In [1]:
%run _standard_imports.ipynb

python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.1.0
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
output_dir = '/lustre/scratch109/malaria/rp7/data/ppq/breakpoint_spanning_reads'


In [6]:
breakpoint_sequences = collections.OrderedDict()
breakpoint_sequences['0bp'] = 'GATAATCACAC'
breakpoint_sequences['1bp'] = 'CGATAATCACACT'
breakpoint_sequences['5bp'] = 'ATTACGATAATCACACTGTTG'
breakpoint_sequences['10bp'] = 'TTATGATTACGATAATCACACTGTTGGTTTC'
breakpoint_sequences['15bp'] = 'ACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCT'
breakpoint_sequences['20bp'] = 'ATTTTACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCTTGCCA'
breakpoint_sequences['5bp']

'ATTACGATAATCACACTGTTG'

In [8]:
from Bio.Seq import Seq
print(Seq(breakpoint_sequences['1bp']).reverse_complement())


AGTGTGATTATCG


In [50]:
tbl_breakpoint_reads.totsv("%s/plasmepsin_1_3.tsv" % output_dir)

In [3]:
# Note we used same samples as were previously used for HRP deletions. See 20160621_HRP_sample_metadata.ipynb
manifest_fn = '/nfs/team112_internal/rp7/data/Pf/hrp/metadata/hrp_manifest_20160621.txt'


In [5]:
tbl_manifest = etl.fromtsv(manifest_fn)
print(len(tbl_manifest.data()))
tbl_manifest

7979


path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/a/1/d/0/265637/1_bam_merge/pe.1.bam,QT0001-C,CM001-J0,CM001,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22429396,110.27,22866093,21829039,20453074
/lustre/scratch109/malaria/pfalciparum/output/f/1/2/c/265638/1_bam_merge/pe.1.bam,QT0001-CW4,CM001-H8,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21661068,96.55,22734327,20463738,18329425
/lustre/scratch109/malaria/pfalciparum/output/0/2/0/6/265639/1_bam_merge/pe.1.bam,QT0001-CW5,CM001-H16,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21409846,68.65,22651241,19897257,17210983
/lustre/scratch109/malaria/pfalciparum/output/d/a/a/f/265640/1_bam_merge/pe.1.bam,QT0001-CW6,CM001-H24,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,6065338,4.34,16902776,1391631,106667
/lustre/scratch109/malaria/pfalciparum/output/6/7/b/4/265641/1_bam_merge/pe.1.bam,QT0002-C,CM002-J0,CM002,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22434258,92.75,22854818,21789730,20157016


In [10]:
import stat
bsub = sh.Command('bsub')

for breakpoint_sequence_name in breakpoint_sequences:
# for breakpoint_sequence_name in ['10bp']:
    breakpoint_reads_dir = '%s/plasmepsin_1_3_7979_samples/%s' % (output_dir, breakpoint_sequence_name)
    !mkdir -p {breakpoint_reads_dir}/results
    !mkdir -p {breakpoint_reads_dir}/scripts
    !mkdir -p {breakpoint_reads_dir}/logs

    breakpoint_sequence = "'%s|%s'" % (
        breakpoint_sequences[breakpoint_sequence_name],
        Seq(breakpoint_sequences[breakpoint_sequence_name]).reverse_complement()
    )
    for rec in tbl_manifest:
        bam_fn = rec[0]
        ox_code = rec[1]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        if not os.path.exists(num_breakpoint_reads_fn):
            script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
            fo = open(script_fn, 'w')
            print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
            fo.close()
            st = os.stat(script_fn)
            os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
            bsub(
                '-G', 'malaria-dk',
                '-P', 'malaria-dk',
                '-q', 'normal',
                '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
                '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
                '-J', 'nbpr_%s' % (ox_code),
                '-R', "'select[mem>4000] rusage[mem=4000]'",
                '-M', '4000',
                script_fn
            )



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [13]:
tbl_manifest

path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/a/1/d/0/265637/1_bam_merge/pe.1.bam,QT0001-C,CM001-J0,CM001,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22429396,110.27,22866093,21829039,20453074
/lustre/scratch109/malaria/pfalciparum/output/f/1/2/c/265638/1_bam_merge/pe.1.bam,QT0001-CW4,CM001-H8,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21661068,96.55,22734327,20463738,18329425
/lustre/scratch109/malaria/pfalciparum/output/0/2/0/6/265639/1_bam_merge/pe.1.bam,QT0001-CW5,CM001-H16,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21409846,68.65,22651241,19897257,17210983
/lustre/scratch109/malaria/pfalciparum/output/d/a/a/f/265640/1_bam_merge/pe.1.bam,QT0001-CW6,CM001-H24,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,6065338,4.34,16902776,1391631,106667
/lustre/scratch109/malaria/pfalciparum/output/6/7/b/4/265641/1_bam_merge/pe.1.bam,QT0002-C,CM002-J0,CM002,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22434258,92.75,22854818,21789730,20157016


In [5]:
pf_5_0_breakpoint_reads = collections.OrderedDict()

# for breakpoint_sequence_name in breakpoint_sequences:
for breakpoint_sequence_name in ['10bp']:
    pf_5_0_breakpoint_reads[breakpoint_sequence_name] = collections.OrderedDict()
    breakpoint_reads_dir = '%s/plasmepsin_1_3_pf_5_0/%s' % (output_dir, breakpoint_sequence_name)
    for rec in tbl_5_0_manifest:
        bam_fn = rec[0]
        ox_code = rec[2]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        fi = open(num_breakpoint_reads_fn, 'r')
        pf_5_0_breakpoint_reads[breakpoint_sequence_name][ox_code] = int(fi.read())



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [15]:
breakpoint_reads_dict = collections.OrderedDict()

for breakpoint_sequence_name in breakpoint_sequences:
    print(breakpoint_sequence_name)
# for breakpoint_sequence_name in ['10bp']:
    breakpoint_reads_dict[breakpoint_sequence_name] = collections.OrderedDict()
    breakpoint_reads_dir = '%s/plasmepsin_1_3_7979_samples/%s' % (output_dir, breakpoint_sequence_name)
    for rec in tbl_manifest.data():
        bam_fn = rec[0]
        ox_code = rec[1]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        fi = open(num_breakpoint_reads_fn, 'r')
        breakpoint_reads_dict[breakpoint_sequence_name][ox_code] = int(fi.read())



0bp
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [17]:
tbl_breakpoint_reads = (etl.wrap(zip(breakpoint_reads_dict['0bp'].keys(), breakpoint_reads_dict['0bp'].values())).pushheader(['ox_code', 'bp_reads_0bp'])
 .join(
        etl.wrap(zip(breakpoint_reads_dict['1bp'].keys(), breakpoint_reads_dict['1bp'].values())).pushheader(['ox_code', 'bp_reads_1bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['5bp'].keys(), breakpoint_reads_dict['5bp'].values())).pushheader(['ox_code', 'bp_reads_5bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['10bp'].keys(), breakpoint_reads_dict['10bp'].values())).pushheader(['ox_code', 'bp_reads_10bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['15bp'].keys(), breakpoint_reads_dict['15bp'].values())).pushheader(['ox_code', 'bp_reads_15bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['20bp'].keys(), breakpoint_reads_dict['20bp'].values())).pushheader(['ox_code', 'bp_reads_20bp']),
        key='ox_code')
)
# tbl_breakpoint_reads.displayall()

In [21]:
tbl_breakpoint_reads.selectgt('bp_reads_10bp', 0).displayall()

ox_code,bp_reads_0bp,bp_reads_1bp,bp_reads_5bp,bp_reads_10bp,bp_reads_15bp,bp_reads_20bp
PD0033-C,1691,12,1,1,1,1
PD0667-C,479,71,63,50,39,34
PD0727-C,349,57,54,49,42,35
PD0729-C,195,23,19,17,14,13
PD1034-C,157,4,4,4,3,1
PD1407-C,299,38,33,32,30,25
PF0117-C,948,3,1,1,1,1
PF0364-C,44,2,2,2,1,0
PH0041-C,1843,327,293,237,176,87
PH0044-C,1368,210,184,156,112,86


In [22]:
tbl_breakpoint_reads.totsv("%s/plasmepsin_1_3_7979_samples.tsv" % output_dir)

In [23]:
tbl_breakpoint_reads.toxlsx("%s/plasmepsin_1_3_7979_samples.xlsx" % output_dir)